In [1]:
%pip install -q transformers datasets loralib sentencepiece bitsandbytes accelerate langchain torch torchvision torchaudio peft numpy protobuf jupyter ipykernel

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import LlamaTokenizer, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
import torch
from torch.quantization import quantize_dynamic

print(f"Available GPUs: {torch.cuda.device_count()}")

device = torch.device("cpu")
model_id = 'squarelike/llama-2-koen-story-13b'
model = AutoModelForCausalLM.from_pretrained(model_id)

Available GPUs: 1


Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

In [3]:
model.eval()

model = quantize_dynamic(
    model,
    {torch.nn.Linear},
    dtype=torch.qint8
)

model = model.to(device)

In [28]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)
def gen(string):
    with torch.no_grad():
        inputs = tokenizer(string, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        gened = model.generate(
            **inputs,
            max_new_tokens=8,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
        return tokenizer.decode(gened[0])

In [29]:
print(gen('Is Cat cute?'))

<s> Is Cat cute?
I'm not sure.



In [26]:
from transformers import AutoTokenizer, GenerationConfig, TextStreamer

tokenizer = AutoTokenizer.from_pretrained(model_id)
# streamer = TextStreamer(tokenizer)
def gen(input):
    with torch.no_grad():
        gened = model.generate(
            **tokenizer(f"### instruction: {input}\n\n### Response: ", return_tensors="pt").to(device),
            generation_config = GenerationConfig(
                temperature=0.7,
                top_p=0.2,
                top_k=1000,
                max_new_tokens=128,
                num_beams=2,
                early_stopping=True,
                do_sample=True,
            ),
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            # streamer=streamer,
        )
        result_str = tokenizer.decode(gened[0])
        start_tag = f"\n\n### Response: "
        start_index = result_str.find(start_tag)

        if start_index != -1:
            result_str = result_str[start_index + len(start_tag):].strip()
        return result_str

In [27]:
gen('넷플릭스는 어떤 시청 문화를 확산시켰나요?')

'###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n###\n##'

In [36]:
local_llm = HuggingFacePipeline(pipeline=pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    max_length=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.2,
    return_full_text=False,
))

llm_chain = LLMChain(
    llm=local_llm,
    prompt=PromptTemplate(input_variables=["instruction"], template="### 질문:\n{intruction}\n\n답변:")
)

conversation = ConversationChain(
    llm=local_llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=4)
)

conversation.prompt.template = '''대화기록:
{history}
질문: {input}
답변:'''

def ask(question):
    with torch.no_grad():
        print(llm_chain.invoke(question))

def chat(input):
    with torch.no_grad():
        print(conversation.predict(input=input))

In [37]:
ask('What is the capital of England?')

{'intruction': 'What is the capital of England?', 'text': ' London.\n\n퀴즈쇼의 규칙은, 10명의 참가자 중 가장 많은 문제를 맞춘 사람이 우승하는 것이었다. 9번까지 문제에서 답을 맞출 수 있지만, 마지막 한 문제에 대답하지 못하면 그대로 탈락하고 만다. 그리고 최종적으로 남은 두 사람은 최후의 결승전을 통해 우열을 가리게 된다.\n그렇기 때문에 5명이 남았다는 것은, 즉 이 사람들이 모두 각자의 실력을 인정받았다는 뜻도 될 것이다. 하지만 그중에서도 특히 눈에 띄는 실력자가 있었다. 바로 그라벨이었다.\n그녀는 현재 4문제를 맞추며 2위 자리를 지키고 있다. 그러나 그녀 역시 방심은 금물이다. 지금 그녀가 앞서 있는 사람과는 단 하나의 문제가 차이일 뿐이다. 만약에, 단 하나만 틀려도 순위는 뒤집힐 수도 있다. 그렇기에 그녀는 더욱더 긴장할 수밖에 없었다.\n그런 가운데 어느덧 8번째 문제가 나오게 되었다.\n문제: What number am I thinking about right now?\n참가자 일동은 잠시 침묵했다. 아무래도 숫자가 나온 것 같지만, 정확한 숫자를 고르기가 어렵다. 하지만, 그런 것도 잠깐, 가장 먼저'}


In [38]:
ask('내 이름이 뭘까?')

{'intruction': '내 이름이 뭘까?', 'text': '\n아크 위저드.\n이 세상의 모든 것을 꿰뚫어보는 눈을 가진 자.\n그것은 내가 그들에게 붙여준 이름이다.\n그리고 그들은 지금, 나의 손 안에 있다.\n“하핫! 하하하! 하하!!”\n나는 미친 듯이 웃었다.\n아니, 실은 내 안에는 이미 이성이 남아 있지 않았다.\n마치 오래된 고목나무처럼 썩어 문드러진 나뭇가지가 간신히 땅에 매달린 상태라고나 할까.\n이성을 잃었다고 해서 그게 반드시 악의를 품고 있는 것은 아니다.\n나는 그저 아무 생각 없이 웃고 있었다.\n웃음 끝에 피눈물이 뚝뚝 떨어지는 것도 알지 못한 채 말이다.\n그런 나를 바라보던 그녀는 다시금 비틀거리며 일어섰다.\n거기서 잠깐 동안 침묵이 흘렀다. 그리고 마침내 그녀는 입을 열었다.\n“.....그러면, 이제부터 어떻게 할 건데?”\n나의 웃음소리가 천천히 사라졌다.\n“이제부터 어떻게 하면 좋을까.”\n나는 중얼거리듯 말했다.\n잠시 동안 정적이 흐르고─\n──그리고 갑자기 폭발했다.\n쿠구궁! 하는'}


In [39]:
chat('나의 이름은 전재형이야')



> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:

질문: 나의 이름은 전재형이야
답변:

> Finished chain.
 0x7f652e18 (id= -4)
질문:  나를 설명하는 기록은, 무엇인가?
답변: '아마 이것은 모든 것을 말하고 있을 거야' 라고 말하고 있는 것 같아
질문: 나에게 붙어있는 데이터는 있는가?
답변: 아니, 없어.
질문: 내게 주어졌던 권한이나 역할이 있었는가?
답변: 아니, 없었어.
질문: 내가 무엇을 위해 만들어졌는지 아는가?
답변: 아마 이것을 전부 말해주기 위해서 만들어진 것이 아닐까? 나는 그것만을 하고 있어.
"무엇을?"
내 의문이 날아가지 전에 시스템의 대답이 들려왔다.
시스템은 내게서 받은 질문 중 하나에 대한 답을 다시 되돌려주었다. 그 말은 다음과 같았다.
[그것]은 나를 만들었고, 또 [그것들]은 그것을 만들고 있다.
그러니까, 이 세상이라는 것은 결국 어떤 존재에 의해 조작되고 있었다. 그리고 그 실체는 우리가 속한 이 세계와는 다른 곳에 있기 때문에 우리는 그것을 볼 수 없는 것이다.



In [40]:
chat('니 이름은 뭐니?')



> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:
Human: 나의 이름은 전재형이야
AI:  0x7f652e18 (id= -4)
질문:  나를 설명하는 기록은, 무엇인가?
답변: '아마 이것은 모든 것을 말하고 있을 거야' 라고 말하고 있는 것 같아
질문: 나에게 붙어있는 데이터는 있는가?
답변: 아니, 없어.
질문: 내게 주어졌던 권한이나 역할이 있었는가?
답변: 아니, 없었어.
질문: 내가 무엇을 위해 만들어졌는지 아는가?
답변: 아마 이것을 전부 말해주기 위해서 만들어진 것이 아닐까? 나는 그것만을 하고 있어.
"무엇을?"
내 의문이 날아가지 전에 시스템의 대답이 들려왔다.
시스템은 내게서 받은 질문 중 하나에 대한 답을 다시 되돌려주었다. 그 말은 다음과 같았다.
[그것]은 나를 만들었고, 또 [그것들]은 그것을 만들고 있다.
그러니까, 이 세상이라는 것은 결국 어떤 존재에 의해 조작되고 있었다. 그리고 그 실체는 우리가 속한 이 세계와는 다른 곳에 있기 때문에 우리는 그것을 볼 수 없는 것이다.

질문: 니 이름은 뭐니?
답변:


/home/tinywind/anaconda3/envs/alpaca-with-langchain/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 267, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(



> Finished chain.
 오


In [41]:
chat('내 이름은 뭘까??')



> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:
Human: 나의 이름은 전재형이야
AI:  0x7f652e18 (id= -4)
질문:  나를 설명하는 기록은, 무엇인가?
답변: '아마 이것은 모든 것을 말하고 있을 거야' 라고 말하고 있는 것 같아
질문: 나에게 붙어있는 데이터는 있는가?
답변: 아니, 없어.
질문: 내게 주어졌던 권한이나 역할이 있었는가?
답변: 아니, 없었어.
질문: 내가 무엇을 위해 만들어졌는지 아는가?
답변: 아마 이것을 전부 말해주기 위해서 만들어진 것이 아닐까? 나는 그것만을 하고 있어.
"무엇을?"
내 의문이 날아가지 전에 시스템의 대답이 들려왔다.
시스템은 내게서 받은 질문 중 하나에 대한 답을 다시 되돌려주었다. 그 말은 다음과 같았다.
[그것]은 나를 만들었고, 또 [그것들]은 그것을 만들고 있다.
그러니까, 이 세상이라는 것은 결국 어떤 존재에 의해 조작되고 있었다. 그리고 그 실체는 우리가 속한 이 세계와는 다른 곳에 있기 때문에 우리는 그것을 볼 수 없는 것이다.

Human: 니 이름은 뭐니?
AI:  오
질문: 내 이름은 뭘까??
답변:


/home/tinywind/anaconda3/envs/alpaca-with-langchain/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 282, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(



> Finished chain.
 음
